<a href="https://colab.research.google.com/github/dqj5182/Sparse_COO_Tensor_Multiplication_Pytorch/blob/main/SCTM_colab_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**This function is based on PyTorch**

In [1]:
import torch
import numpy as np

### **Sparse Tensor (dot product) Multiplication Function** </br>
*This function allows us to do tensor matrix multiplication for two sparse coo tensors. In other words, it is like doing torch.matmul with two sparse coo tensors, outputting one sparse coo tensor. </br>*
Input: Two Sparse COO Tensors </br>
Output: One Sparse COO Tensor (dot product result of two sparse COO tensor)
*There were no sparse to dense convert

In [2]:
def sparse_coo_mul(tensor1, tensor2):
  tensor1_indices = tensor1.coalesce().indices()
  tensor2_indices = tensor2.coalesce().indices()
  tensor1_values = tensor1.coalesce().values()
  tensor2_values = tensor2.coalesce().values()

  def swap_two_rows(tensor):
    index = [1, 0]
    tensor[index,] = tensor.clone()
    return tensor

  test_1 = tensor1_indices.t()
  test_2 = swap_two_rows(tensor2_indices).t()

  mul_i_j = []

  final = {}

  final_list = []

  for i, val_i in enumerate(test_1):
    for j, val_j in enumerate(test_2):

      if torch.equal(val_i[1], val_j[1]):
        mul_i_j.append([val_i[0].item(), val_j[0].item(), (tensor1_values[i]*tensor2_values[j]).item()]) # val_i[0]: row 1, val_i[1]: col 1, val_j[0]: col 2, val_j[1]: row 2

  for ele1, ele2, ele3 in mul_i_j:

    if (ele1, ele2) not in final:
      final[(ele1, ele2)] = ele3
    else:
      final[(ele1, ele2)] = final[(ele1, ele2)] + ele3

  for key, value in final.items():
    temp = [key[0], key[1] ,value]
    final_list.append(temp)

  final_list = np.array(final_list).T

  new_index = final_list[0:2,]
  new_value = final_list[2,]

  new_sparse_coo_tensor = torch.sparse_coo_tensor(new_index, new_value)
  
  return new_sparse_coo_tensor

### **Testing with two sparse COO tensors** </br>

In [3]:
i_1 = torch.tensor([[0, 1, 1],
                  [2, 1, 2]])
v_1 = torch.tensor([3, 4, 5], dtype=torch.float32)

a = torch.sparse_coo_tensor(i_1, v_1)
a

tensor(indices=tensor([[0, 1, 1],
                       [2, 1, 2]]),
       values=tensor([3., 4., 5.]),
       size=(2, 3), nnz=3, layout=torch.sparse_coo)

In [4]:
i_2 = torch.tensor([[2, 1, 1],
                  [1, 0, 1]])
v_2 = torch.tensor([1, 7, 2], dtype=torch.float32)

b = torch.sparse_coo_tensor(i_2, v_2)
b

tensor(indices=tensor([[2, 1, 1],
                       [1, 0, 1]]),
       values=tensor([1., 7., 2.]),
       size=(3, 2), nnz=3, layout=torch.sparse_coo)

In [5]:
sparse_coo_mul(a, b)

tensor(indices=tensor([[0, 1, 1],
                       [1, 0, 1]]),
       values=tensor([ 3., 28., 13.]),
       size=(2, 2), nnz=3, dtype=torch.float64, layout=torch.sparse_coo)

###**Comparing results with matmul**

1. Make sparse tensor a and b as dense tensor
2. Use torch.matmul to multiply those two tensors

In [6]:
torch.matmul(a.to_dense(), b.to_dense())

tensor([[ 0.,  3.],
        [28., 13.]])

1. Use sparse_coo_mul(a, b) to multiply two sparse tensor
2. Make the multiplication result into dense tensor for easy comparison

In [7]:
sparse_coo_mul(a, b).to_dense()

tensor([[ 0.,  3.],
        [28., 13.]], dtype=torch.float64)

Original sparse coo tensor output for reference

In [8]:
sparse_coo_mul(a, b)

tensor(indices=tensor([[0, 1, 1],
                       [1, 0, 1]]),
       values=tensor([ 3., 28., 13.]),
       size=(2, 2), nnz=3, dtype=torch.float64, layout=torch.sparse_coo)